In [4]:
!bash --login -c "poetry update"

Updating dependencies
Resolving dependencies... (13.9s)

No dependencies to install or update


# Option valuation model based on historical prices

This code assumes `aws configure` has been run in the environment of the notebook.

In [1]:
import optionspricing
import binanceprices
import boto3
COUNT_YEARS = 6
BINANCE_DATETIME_FORMAT = "%Y-%m-%d %H-%M-%S"
BINANCE_SYMBOL = 'ETHUSDT'

s3 = boto3.resource('s3')
prices_df = binanceprices.load_prices(s3, "prod-binance-prices", BINANCE_SYMBOL, COUNT_YEARS)


## Option valuation model: input parameters here

In [2]:
target_period_hours = 3 * 24
strikes_universe_size = 4
CUT_OFF_YEAR_MONTH = (2022, 1)

instrument_code = BINANCE_SYMBOL[:3]

headers = {"Content-Type": "application/json"}
base_url = "https://www.deribit.com/api/v2/public"

trading_model = optionspricing.TradingModel(base_url, headers, instrument_code, target_period_hours)
trading_model.cutoff_year_month(CUT_OFF_YEAR_MONTH)
target_expiry = trading_model.target_expiry
underlying_price = trading_model.underlying_price
remaining_hours = trading_model.remaining_hours

simulation = trading_model.evaluate(prices_df, strikes_universe_size)
print(simulation)
simulation.allocate([1, 2], 1500. * 2./100.)

target expiry: 2024-02-05 08:00:00+00:00 (72 hours left)
current price: 2311.39
-------------------------------
trading put 2275 and call 2325 (hit ratio: 56%)
cost: 0.014 / value: 0.035, benefit/cost = 2.4x
($) cost: 33.52 / value: 80.46, average gain = 46.95
-------------------------------
trading put 2250 and call 2350 (hit ratio: 50%)
cost: 0.009 / value: 0.027, benefit/cost = 3.1x
($) cost: 20.11 / value: 63.11, average gain = 43.00
-------------------------------
trading put 2225 and call 2375 (hit ratio: 43%)
cost: 0.005 / value: 0.021, benefit/cost = 4.0x
($) cost: 12.25 / value: 49.53, average gain = 37.28
-------------------------------
trading put 2200 and call 2400 (hit ratio: 34%)
cost: 0.003 / value: 0.017, benefit/cost = 4.9x
($) cost: 7.86 / value: 38.87, average gain = 31.01


[{'put': 2250.0, 'call': 2350.0, 'quantity': 1},
 {'put': 2225.0, 'call': 2375.0, 'quantity': 1}]

In [5]:
simulation.options_chain

,value_call,value_call_pct,call_ask,value_put,value_put_pct,put_ask
strike,,,,,,
2175.0,104.585638,NaN,NaN,13.248727,0.005846,0.0017
2200.0,84.031209,NaN,NaN,17.694297,0.007808,0.0032
2225.0,65.111732,NaN,NaN,23.774820,0.010491,0.0055
2250.0,48.653754,NaN,0.0165,32.316843,0.014260,0.0090
2275.0,35.764609,0.015782,0.0105,44.427698,NaN,0.0140
2300.0,26.473994,0.011682,0.0065,60.137083,NaN,0.0225
2325.0,19.662775,0.008676,0.0041,78.325864,NaN,0.0320
2350.0,14.653544,0.006466,0.0025,98.316633,NaN,NaN
2375.0,10.958756,0.004836,0.0017,119.621845,NaN,NaN
